## Step 1: Installation and Setup
Make sure you have all the necessary libraries installed. Typically, you can install them via:

```bash
!pip install mlflow scikit-learn boto3 pandas numpy
```

In [ ]:
import os
import warnings
import logging
from urllib.parse import urlparse

import boto3
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

warnings.filterwarnings("ignore")
np.random.seed(40)

## Step 2: Define a helper function
We'll define a helper function `eval_metrics` for evaluating our model predictions.

In [ ]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

## Step 3: Download and Prepare the Data
The wine-quality dataset is publicly available. We read it directly from the URL.


In [ ]:
# Download the dataset
csv_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"

try:
    data = pd.read_csv(csv_url, sep=";")
except Exception as e:
    logger.exception(
        "Unable to download training & test CSV. Check your internet connection. Error: %s", e
    )
    raise e

# Split into train/test sets
train, test = train_test_split(data, test_size=0.25)

train_x = train.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_x = test.drop(["quality"], axis=1)
test_y = test[["quality"]]

print(f"Train set size: {train.shape}")
print(f"Test set size: {test.shape}")

## Step 5: (Optional) Set an MLflow Experiment
If you want to keep your runs organized under a named experiment, 
you can do so here. If the experiment does not exist, MLflow will create it.


In [ ]:
EXPERIMENT_NAME = "WineQuality_ElasticNet"
mlflow.set_experiment(EXPERIMENT_NAME)
print(f"Using MLflow experiment: {EXPERIMENT_NAME}")

## Step 6: Run Multiple Training Sessions
We can loop through various hyperparameter values (e.g., `alpha` and `l1_ratio`) 
and track each run in MLflow. MLflow will automatically capture parameters, 
metrics, and artifacts.

In [ ]:
from mlflow.models.signature import infer_signature

# %%
alpha_values = [0.1, 0.3, 0.5]
l1_values = [0.1, 0.5, 0.9]

example_input = train_x.iloc[:5]  # A small snippet of your training data as an example

for alpha in alpha_values:
    for l1_ratio in l1_values:
        with mlflow.start_run():
            # Train model
            lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
            lr.fit(train_x, train_y)
            predictions = lr.predict(example_input)
            signature = infer_signature(example_input, predictions)


            # Predict
            predicted_qualities = lr.predict(test_x)

            # Evaluate
            (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

            print(f"ElasticNet model (alpha={alpha}, l1_ratio={l1_ratio}):")
            print(f"  RMSE: {rmse}")
            print(f"  MAE: {mae}")
            print(f"  R2: {r2}\n")

            # Log parameters and metrics
            mlflow.log_param("alpha", alpha)
            mlflow.log_param("l1_ratio", l1_ratio)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Log model
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
            if tracking_url_type_store != "file":
                # Register the model if a model registry is available
                mlflow.sklearn.log_model(
                    lr, "model", 
                    registered_model_name="ElasticnetWineModel",
                    input_example=example_input,
                    signature=signature
                )
            else:
                # Otherwise, just log the model locally
                mlflow.sklearn.log_model(lr, "model", input_example=example_input,
                    signature=signature)
            mlflow.end_run()


## Step 7: Model Versioning & Registry
After running multiple trainings, if you are using the MLflow Model Registry 
(e.g., with a supported backend store), you can see different versions of 
`ElasticnetWineModel`.

Use the MLflow UI to compare runs, check metrics, and manage model versions.

You could programmatically transition a model version to staging or production with:


In [ ]:
from mlflow.tracking import MlflowClient
#
client = MlflowClient()
client.transition_model_version_stage(
    name="ElasticnetWineModel", version=1, stage="Staging"
)

#
# (This requires that you have the model registry set up, e.g. a database backend and 
# the MLflow Tracking server.)

## Step 8: Additional Artifacts
You can log additional artifacts such as plots, confusion matrices, or environment files.

For instance:

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.scatter(test_y, predicted_qualities)
ax.set_xlabel("Actual Quality")
ax.set_ylabel("Predicted Quality")
plt.savefig("scatter_plot.png")


# This command logs the artifact to MLFlow. You can find it under "Experiments" > "<Name of the experiment>".
# It will be the last entry in the artifact list
mlflow.log_artifact("scatter_plot.png")

## Summary
In this notebook, we demonstrated:
1. Setting up MLflow tracking in a notebook.
2. Logging parameters, metrics, and models.
3. Tracking multiple runs with different hyperparameters.
4. (Optional) Model versioning using the MLflow Model Registry.
#
# Try exploring your runs, comparing them in the UI, and picking the best model for deployment.